### 1. Importing necessary libraries

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import (
    classification_report,
    f1_score,
    precision_score,
    recall_score
)
from utils.utils import plot_confusion_matrix, generate_tensor

from crack_segmentation.subroutines.loss_metrics import (
    Weighted_Cross_Entropy,
    F1_score,
    Precision_dil,
    F1_score_dil
)

### 2. Loading the dataset

In [2]:
dataset = pd.read_csv('../dataset/dataset.csv')

test_df = dataset.sample(frac=1.0, random_state=42)
test_data = generate_tensor(test_df, image_size=(224,224), class_mode='categorical')

### 3. Loading the Unet-MobileNet model

In [3]:
model = tf.keras.models.load_model(
    'models/trained/unet_mobilenet.h5',
    custom_objects={
        'loss': Weighted_Cross_Entropy(10),
        'F1_score': F1_score,
        'F1_score_dil': F1_score_dil,
        'Precision_dil': Precision_dil
    }
)

model.summary()

### 4. Making predictions

In [4]:
y_pred = np.argmax(model.predict(test_data), axis=-1).astype(np.uint8)

#### 4.1. Classification report

In [5]:
cr = classification_report(test_data.labels, y_pred)

print(cr)

#### 4.2. F1, Precision and Recall Score

In [6]:
x = f1_score(test_data.labels, y_pred,)
y = precision_score(test_data.labels, y_pred)
z = recall_score(test_data.labels, y_pred)

In [7]:
print(f'F1 Score: {round(x*100,2)}%')
print(f'Precision Score: {round(y*100,2)}%')
print(f'Recall Score: {round(z*100,2)}%')

#### 4.3. Confusion matrix

In [8]:
disp = plot_confusion_matrix(test_data.labels, y_pred, title='Unet MobileNet')

plt.show()

### 4. Evaluating the model

In [9]:
loss, acc = model.evaluate(test_data)

In [10]:
print(f'acc: {round(acc*100,2)}%')
print(f'loss: {loss}' )